In [1]:
# project: p12
# submitter: wchen394
# partner: zyu293
# hours: 2

In [2]:
import os, json, csv
from collections import namedtuple
from operator import attrgetter
import copy 
import matplotlib
import pandas as pd
from pandas import DataFrame, Series
import requests   
from bs4 import BeautifulSoup
from math import radians, cos, sin, asin, sqrt
import math

In [3]:
def download(filename, url):
    # We do not download again if the file already exists
    if os.path.exists(filename):
        return (str(filename) + " already exists!")

    # TODO: Write the code to download the file from URL
    # and save it in `filename`
    r = requests.get(url)
    r.raise_for_status
    filename = open(filename, "w", encoding="utf-8")
    filename.write(r.text)
    filename.close()
    return (str(filename) + " created!")

In [4]:
DataFrame([[1,2], [3,4]])

0  1
0  1  2
1  3  4

In [5]:
# option 2
DataFrame([{"x":1, "y":2},
           {"x":3, "y":4}])

# option 3
DataFrame({"x":[1,3],
           "y":[2,4]})

x  y
0  1  2
1  3  4

In [6]:
# option 4
DataFrame({"x":{"A":1,"B":3},
           "y":{"A":2,"B":4}})

x  y
A  1  2
B  3  4

In [7]:
download("countries.json", "https://raw.githubusercontent.com/msyamkumar/cs220-s21-projects/master/p12/countries.json")

'countries.json already exists!'

In [8]:
countries = pd.read_json('countries.json')

In [9]:
#Q1: How many countries do we have in our dataset?
len(countries)

174

In [10]:
#Q2: what is the total population across all the countries in our dataset?
countries["population"].sum()

6261901793

In [11]:
download("capitals.json", "https://raw.githubusercontent.com/msyamkumar/cs220-s21-projects/master/p12/capitals.json")

'capitals.json already exists!'

In [12]:
capitals = pd.read_json('capitals.json')

In [13]:
capitals.head()

country       capital   latitude  longitude
0  Afghanistan         Kabul  34.516667  69.183333
1      Albania        Tirana  41.316667  19.816667
2      Algeria       Algiers  36.750000   3.050000
3     Anguilla    The Valley  18.216667 -63.050000
4    Argentina  Buenos Aires -34.583333 -58.666667

In [14]:
#Q3: What are the capital names in capitals.json?
capital_names = list(capitals["capital"])
capital_names = sorted(capital_names)
capital_names

['Abu Dhabi',
 'Abuja',
 'Accra',
 'Addis Ababa',
 'Algiers',
 'Amman',
 'Amsterdam',
 'Ankara',
 'Antananarivo',
 'Apia',
 'Ashgabat',
 'Asmara',
 'Astana',
 'Asuncion',
 'Athens',
 'Baghdad',
 'Baku',
 'Bamako',
 'Bangkok',
 'Beijing',
 'Beirut',
 'Belmopan',
 'Berlin',
 'Bern',
 'Bishkek',
 'Bissau',
 'Bogota',
 'Brasilia',
 'Bridgetown',
 'Brussels',
 'Bucharest',
 'Budapest',
 'Buenos Aires',
 'Bujumbura',
 'Cairo',
 'Canberra',
 'Caracas',
 'Castries',
 'Chisinau',
 'Colombo',
 'Conakry',
 'Copenhagen',
 'Dakar',
 'Damascus',
 'Dar es Salaam',
 'Dhaka',
 'Djibouti',
 'Doha',
 'Dublin',
 'Dushanbe',
 'Freetown',
 'Gaborone',
 'George Town',
 'Georgetown',
 'Guatemala City',
 'Hagatna',
 'Hamilton',
 'Hanoi',
 'Harare',
 'Havana',
 'Helsinki',
 'Islamabad',
 'Jakarta',
 'Jamestown',
 'Jerusalem',
 'Kabul',
 'Kampala',
 'Kathmandu',
 'Khartoum',
 'Kigali',
 'Kingston',
 'Kingstown',
 'Kuala Lumpur',
 'Kuwait City',
 'Kyiv',
 'La Paz',
 'Libreville',
 'Lilongwe',
 'Lima',
 'Lisbon',


In [15]:
#Q4: What is the capital of Sweden?
capitals[capitals['country'] == "Sweden"]["capital"].iloc[0]

'Stockholm'

In [16]:
#Q5: Which country's capital is Santiago?
capitals[capitals['capital'] == "Santiago"]["country"].iloc[0]

'Chile'

In [17]:
#Q6: Which 7 countries have the southern-most capitals?
data_6 = capitals.sort_values("latitude").head(7).values.tolist()
res=[]
for i in data_6:
    res.append(i[0])
res

['New Zealand',
 'Australia',
 'Uruguay',
 'Argentina',
 'Chile',
 'Lesotho',
 'Swaziland']

In [18]:
#Q7: Which 10 countries have the northern-most capitals?
data_7 = capitals.sort_values("latitude",ascending=False).head(10).values.tolist()
res=[]
for i in data_7:
    res.append(i[0])
res

['Iceland',
 'Finland',
 'Norway',
 'Estonia',
 'Sweden',
 'Latvia',
 'Russia',
 'Denmark',
 'Lithuania',
 'Belarus']

In [19]:
#Q8: What is the largest land-locked country in South America?
filter1 = countries[countries["coastline"]==0]
filter1[filter1["continent"]=="South America"].sort_values("area",ascending=False).head(1)["country"].iloc[0]

'Bolivia'

In [20]:
#Q9: What is the largest coastal country in Africa?
filter1 = countries[countries["coastline"]!=0]
filter1[filter1["continent"]=="Africa"].sort_values("area",ascending=False).head(1)["country"].iloc[0]

'Sudan'

In [21]:
#Q10: What is coastal country with the least population in Asia?
filter1 = countries[countries["coastline"]!=0]
filter1[filter1["continent"]=="Asia"].sort_values("population").head(1)["country"].iloc[0]

'Maldives'

In [22]:
#cite from https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [23]:
#Q11: What is the distance between Camp Randall Stadium and the Wisconsin State Capital?
haversine(-89.411893,43.070231,-89.384113,43.074645)

1.433899492072933

In [24]:
#Q12: What is the distance between Belgium and France?
la1 = capitals[capitals["country"]=="Belgium"]["latitude"].iloc[0]
lo1 = capitals[capitals["country"]=="Belgium"]["longitude"].iloc[0]
la2 = capitals[capitals["country"]=="France"]["latitude"].iloc[0]
lo2 = capitals[capitals["country"]=="France"]["longitude"].iloc[0]
haversine(lo1,la1,lo2,la2)

162.36419849726036

In [25]:
def get_distance_df(df):
    tmp = {}
    for i in df["country"]:
        cur ={}    
        for j in df["country"]:
        
            value = 0
            if df[df["country"]==i]["country"].iloc[0] == df[df["country"]==j]["country"].iloc[0]:
                value = math.nan
            else:
                value = haversine(capitals[capitals["country"]==i]["longitude"],capitals[capitals["country"]==i]["latitude"],capitals[capitals["country"]==j]["longitude"],capitals[capitals["country"]==j]["latitude"])
            cur[df[df["country"]==j]["country"].iloc[0]]=value
            tmp[df[df["country"]==i]["country"].iloc[0]]=cur
#     print(tmp)
    return DataFrame(tmp)

In [26]:
#Q13: What are the distances between Belgium, France and Spain?
df = countries[(countries["country"]=="Belgium") | (countries["country"]=="France") | (countries["country"]=="Spain")]
get_distance_df(df)


Spain     Belgium      France
Spain           NaN  816.692445  654.572177
Belgium  816.692445         NaN  162.364198
France   654.572177  162.364198         NaN

In [27]:
#Q14: What is the distance between every pair of countries in the continent Australia?
Australia = countries[countries["continent"]=="Australia"]
q14_distance = get_distance_df(Australia)
q14_distance

Marshall Islands  French Polynesia  New Zealand  \
Marshall Islands               NaN       3159.856137  3348.725378   
French Polynesia       3159.856137               NaN  2668.807328   
New Zealand            3348.725378       2668.807328          NaN   
Vanuatu                1727.328621       2765.339382  1671.641167   
Palau                  2516.301728       5447.018761  4218.188412   
Papua New Guinea       2019.144119       4265.757856  2767.607095   
Guam                   1861.584428       4961.131713  4239.314912   
Samoa                  1849.150692       1496.953816  2062.548020   
New Caledonia          2054.905160       2864.497308  1400.483623   
Australia              3263.901948       3920.570216  1445.147895   
Fiji                   1806.972599       2102.135722  1613.916992   
Tonga                  2151.129410       1686.539411  1510.219450   

                      Vanuatu        Palau  Papua New Guinea         Guam  \
Marshall Islands  1727.328621  2516.301728       2019.144119  1861.584428   
French Polynesia  2765.339382  5447.018761       4265.757856  4961.131713   
New Zealand       1671.641167  4218.188412       2767.607095  4239.314912   
Vanuatu                   NaN  2882.041892       1527.495923  2687.395771   
Palau             2882.041892          NaN       1453.306058   800.243081   
Papua New Guinea  1527.495923  1453.306058               NaN  1591.171595   
Guam              2687.395771   800.243081       1591.171595          NaN   
Samoa             1349.904435  3956.305436       2789.123270  3520.883355   
New Caledonia      335.602971  2974.892838       1552.708901  2871.868601   
Australia         1687.810709  3099.023227       1786.749660  3377.103870   
Fiji               663.967196  3468.037102       2174.712829  3165.570014   
Tonga             1098.016273  3931.843110       2623.742354  3617.512808   

                        Samoa  New Caledonia    Australia         Fiji  \
Marshall Islands  1849.150692    2054.905160  3263.901948  1806.972599   
French Polynesia  1496.953816    2864.497308  3920.570216  2102.135722   
New Zealand       2062.548020    1400.483623  1445.147895  1613.916992   
Vanuatu           1349.904435     335.602971  1687.810709   663.967196   
Palau             3956.305436    2974.892838  3099.023227  3468.037102   
Papua New Guinea  2789.123270    1552.708901  1786.749660  2174.712829   
Guam              3520.883355    2871.868601  3377.103870  3165.570014   
Samoa                     NaN    1542.152644  2840.091065   716.316054   
New Caledonia     1542.152644            NaN  1375.924712   825.890876   
Australia         2840.091065    1375.924712          NaN  2145.768971   
Fiji               716.316054     825.890876  2145.768971          NaN   
Tonga              553.393188    1179.069567  2362.766922   463.843872   

                        Tonga  
Marshall Islands  2151.129410  
French Polynesia  1686.539411  
New Zealand       1510.219450  
Vanuatu           1098.016273  
Palau             3931.843110  
Papua New Guinea  2623.742354  
Guam              3617.512808  
Samoa              553.393188  
New Caledonia     1179.069567  
Australia         2362.766922  
Fiji               463.843872  
Tonga                     NaN

In [28]:
Australia

area  birth-rate  coastline  continent           country  death-rate  \
4      11854       33.05       3.12  Australia  Marshall Islands        4.78   
25      4167       16.68      60.60  Australia  French Polynesia        4.69   
53    268680       13.76       5.63  Australia       New Zealand        7.53   
56     12200       22.72      20.72  Australia           Vanuatu        7.82   
58       458       18.03     331.66  Australia             Palau        6.80   
74    462840       29.36       1.11  Australia  Papua New Guinea        7.25   
110      541       18.79      23.20  Australia              Guam        4.48   
117     2944       16.43      13.69  Australia             Samoa        6.62   
137    19060       18.11      11.83  Australia     New Caledonia        5.69   
149  7686850       12.14       0.34  Australia         Australia        7.51   
160    18270       22.55       6.18  Australia              Fiji        5.65   
171      748       25.37      56.02  Australia             Tonga        5.28   

     gdp-per-capita  infant-mortality literacy  net-migration phones  \
4              1600             29.45     93,7          -6.04   91,2   
25            17500              8.44     98,0           2.94  194,5   
53            21600              5.85     99,0           4.05  441,7   
56             2900             55.16     53,0           0.00   32,6   
58             9000             14.84     92,0           2.85  325,6   
74             2200             51.45     64,6           0.00   10,9   
110           21000              6.94     99,0           0.00  492,0   
117            5600             27.71     99,7         -11.70   75,2   
137           15000              7.72     91,0           0.00  252,2   
149           29000              4.69    100,0           3.98  565,5   
160            5800             12.62     93,7          -3.14  112,6   
171            2200             12.62     98,5           0.00   97,7   

     population   region  
4         60422  OCEANIA  
25       274578  OCEANIA  
53      4076140  OCEANIA  
56       208869  OCEANIA  
58        20579  OCEANIA  
74      5670544  OCEANIA  
110      171019  OCEANIA  
117      176908  OCEANIA  
137      219246  OCEANIA  
149    20264082  OCEANIA  
160      905949  OCEANIA  
171      114689  OCEANIA

In [29]:
#Q15: What is the most central country in the continent Australia?
q14_distance.mean().sort_values().index[0]

'Vanuatu'

In [30]:
#Q16: What is the least central country in the continent Australia?
q14_distance.mean().sort_values(ascending=False).index[0]

'French Polynesia'

In [31]:
#Q17: How close is each country in the continent Australia to its nearest neighbor?
value = q14_distance.min()
place = q14_distance.idxmin()
res = {}
for i in range(2):
    for j in range(len(q14_distance)):
        cur = {}
        cur["nearest"]=place[j]
        cur["distance"]=value[j]
        res[Australia["country"].iloc[j]] = cur

df = DataFrame(res)
df = df.T
df

nearest distance
Marshall Islands        Vanuatu  1727.33
French Polynesia          Samoa  1496.95
New Zealand       New Caledonia  1400.48
Vanuatu           New Caledonia  335.603
Palau                      Guam  800.243
Papua New Guinea          Palau  1453.31
Guam                      Palau  800.243
Samoa                     Tonga  553.393
New Caledonia           Vanuatu  335.603
Australia         New Caledonia  1375.92
Fiji                      Tonga  463.844
Tonga                      Fiji  463.844

In [32]:
#Q18: How far is each country in the Australia continent to its furthest neighbor?
value = q14_distance.max()
place = q14_distance.idxmax()
res = {}
for i in range(len(q14_distance)):
    cur = {}
    cur["furthest"]=place[i]
    cur["distance"]=value[i]
    res[Australia["country"].iloc[i]] = cur

df = DataFrame(res)
df = df.T
df

furthest distance
Marshall Islands       New Zealand  3348.73
French Polynesia             Palau  5447.02
New Zealand                   Guam  4239.31
Vanuatu                      Palau  2882.04
Palau             French Polynesia  5447.02
Papua New Guinea  French Polynesia  4265.76
Guam              French Polynesia  4961.13
Samoa                        Palau  3956.31
New Caledonia                Palau  2974.89
Australia         French Polynesia  3920.57
Fiji                         Palau  3468.04
Tonga                        Palau  3931.84

In [33]:
#Q19: For birth-rate and death-rate, what are various summary statistics (e.g., mean, max, standard deviation, etc.)?
countries.describe()

area  birth-rate   coastline  death-rate  gdp-per-capita  \
count  1.740000e+02  174.000000  174.000000  174.000000      174.000000   
mean   7.308669e+05   22.463851   10.895000    9.625172     9829.885057   
std    2.011115e+06   11.278992   38.347494    5.187143    10624.354342   
min    2.000000e+00    8.250000    0.000000    2.410000      500.000000   
25%    2.777000e+04   12.597500    0.090000    6.027500     1900.000000   
50%    1.375200e+05   20.010000    0.450000    8.230000     5250.000000   
75%    5.244775e+05   29.860000    3.165000   11.715000    14725.000000   
max    1.707520e+07   50.730000  331.660000   29.740000    55100.000000   

       infant-mortality  net-migration    population  
count        174.000000     174.000000  1.740000e+02  
mean          37.749425       0.124425  3.598794e+07  
std           35.242398       4.479222  1.337194e+08  
min            2.290000     -13.920000  7.502000e+03  
25%            8.540000      -0.907500  2.027785e+06  
50%           23.920000       0.000000  7.454650e+06  
75%           61.732500       0.815000  2.287946e+07  
max          163.070000      23.060000  1.313974e+09

In [34]:
def get_valid_countries():
    return capitals["country"].tolist()
def get_data():
    country_list = get_valid_countries()
    download('capitals.html', 'https://raw.githubusercontent.com/msyamkumar/cs220-s21-projects/master/p12/techslides-snapshot.html')
    f = open("capitals.html", encoding="utf-8")
    data = f.read()
    f.close()
    doc = BeautifulSoup(data, "html.parser")
    header = []
    for e in doc.find_all("tr")[0].find_all("td"):
        if e not in ('Country Code', 'Continent Name'):
            header.append(e.get_text())
    row_data = []
    for row in doc.find_all("tr")[1:]:
        single = []
        if row.find_all("td")[0].get_text() not in country_list:
            continue
        for e in row.find_all("td"):
            single.append(e.get_text())
        row_data.append(single)
    res = []
    header = []
    header.append("country")
    header.append("capital")
    header.append("latitude")
    header.append("longitude")
    for row in row_data:
        dic = {}
        for i in range(len(header)):
            if i in (2,3):
                row[i] = float(row[i])
            dic[header[i]] = row[i]
        res.append(dic)
    return res

In [35]:
#Q20: BeautifulSoup
# my_capitals.json = json.dump(get_data())
dic = get_data()
out_file = open("my_capitals.json", "w")
json.dump(dic, out_file, indent = 6)
out_file.close()
f = open('my_capitals.json', encoding='utf-8')
my_capitals_data = json.load(f)
f.close()
# DataFrame(my_capitals_data).values.tolist()
res =  []
data_20 = DataFrame(my_capitals_data).values.tolist()
# for i in data_20:
#     cur = []
#     ind = 0
#     for j in i:
#         if ind != 0:
#             cur.append(j)
#         ind += 1
#     dic = {}
#     dic[i[0]] = cur
#     res.append(dic)
# res
my_capitals_data

[{'country': 'Afghanistan',
  'capital': 'Kabul',
  'latitude': 34.51666667,
  'longitude': 69.183333},
 {'country': 'Albania',
  'capital': 'Tirana',
  'latitude': 41.31666667,
  'longitude': 19.816667},
 {'country': 'Algeria',
  'capital': 'Algiers',
  'latitude': 36.75,
  'longitude': 3.05},
 {'country': 'Anguilla',
  'capital': 'The Valley',
  'latitude': 18.21666667,
  'longitude': -63.05},
 {'country': 'Argentina',
  'capital': 'Buenos Aires',
  'latitude': -34.58333333,
  'longitude': -58.666667},
 {'country': 'Armenia',
  'capital': 'Yerevan',
  'latitude': 40.16666667,
  'longitude': 44.5},
 {'country': 'Aruba',
  'capital': 'Oranjestad',
  'latitude': 12.51666667,
  'longitude': -70.033333},
 {'country': 'Australia',
  'capital': 'Canberra',
  'latitude': -35.26666667,
  'longitude': 149.133333},
 {'country': 'Austria',
  'capital': 'Vienna',
  'latitude': 48.2,
  'longitude': 16.366667},
 {'country': 'Azerbaijan',
  'capital': 'Baku',
  'latitude': 40.38333333,
  'longitude'